In [62]:

import pandas as pd
import os
import configparser
import datetime as dt

In [28]:
config = configparser.ConfigParser()
config.read('./Data/estados_de_cuenta/config.ini')
data_address=config['DATA']['ADDRESS']


In [127]:
card_type = 'dc' #cc for credit card, dc for debit card
card_csv_files = [file_name for file_name in os.listdir(data_address+'/cards') if card_type in file_name]

In [136]:
#CREDIT CARD USE ANALYSIS

#Define columns names and type to use during csv import
colums = {"date":str,"description":str,"amount":float}
transactions_df = pd.DataFrame()
#Create lambda function to import csv file. Input is file name
import_csv_files = lambda card_file_name: pd.read_csv(f"{data_address}/cards/{card_file_name}",names=colums.keys(),thousands=',',dtype=colums)
csv_dataframes = []
for file in card_csv_files:
    csv_dataframes.append(import_csv_files(file))
card_transaction_history = pd.concat(csv_dataframes).drop_duplicates()
#card_transaction_history = card_transaction_history[card_transaction_history["amount"]>=0]
card_transaction_history["date"] = pd.to_datetime(card_transaction_history["date"],format="%d/%m/%Y")
card_transaction_history.sort_values(by="date")


,date,description,amount
60,2024-07-01,ADO TEZIUTLAN 0200624,216.00
61,2024-07-01,GASOLINA CARROS 0190624,794.58
55,2024-07-01,CGO Clases y entrenamiento,-800.00
56,2024-07-01,PAGO DE TARJETA: 5470748018127633 EN BPI,-2500.00
59,2024-07-01,GASOLINA CAMIONETA 0220624,616.49
...,...,...,...
3,2024-10-14,GASOLINA 41024 0241014,454.48
2,2024-10-14,GASOLINA 051024 0241014,903.48
1,2024-10-14,INTERNET CASA 0241014,600.00
17,2024-10-14,LC102 PIB 28734 04246PFA940043931293 14,-70.00


In [145]:
month_to_analyse = 10
print(card_transaction_history[card_transaction_history['date'].dt.month == month_to_analyse].sort_values(by="amount",ascending=False))
sum(card_transaction_history[card_transaction_history['date'].dt.month == month_to_analyse].amount)

         date                               description    amount
21 2024-10-07  INSTRUYE IRVTUS3NXXX CAMPO 72 SI 0782145  10507.05
20 2024-10-08  SERVICIO CAMIONETA               0309240   6500.00
7  2024-10-14  COMIDA PAPAS VENA CAVA           0241014   1045.00
14 2024-10-14  GASOLINA 040924                  0241014   1022.39
2  2024-10-14  GASOLINA 051024                  0241014    903.48
16 2024-10-14  GASOLINA 290824                  0241014    762.16
9  2024-10-14  GASOLINA 170924                  0241014    716.58
13 2024-10-14  COMIDA SUSHI                     0241014    633.00
1  2024-10-14  INTERNET CASA                    0241014    600.00
11 2024-10-14  GASOLINA 120924                  0241014    599.51
5  2024-10-14  GASOLINA 280924                  0241014    589.11
15 2024-10-14  GASOLINA 310824                  0241014    567.12
6  2024-10-14  GASOLINA 260924                  0241014    554.70
12 2024-10-14  INTERNET CASA                    0241014    550.00
3  2024-10

12553.8